# Aldar Köse Storyboard Generator - Google Colab Setup

This notebook runs the image generation backend on Google Colab with GPU acceleration.

**Two modes:**
1. **Full Server Mode**: Run the entire Flask app in Colab (accessible via ngrok)
2. **API Mode**: Expose only the generation API for your local Flask app to call

**Requirements:**
- Google Colab with GPU enabled (Runtime → Change runtime type → GPU)
- Your code repository accessible (GitHub or Drive)
- ngrok account (free) for public URL

## 1. Setup: Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi

print("\n" + "="*70)
print("GPU Available - Ready for fast SDXL generation!")
print("="*70)

In [ ]:
# Install required packages
!pip install -q flask python-dotenv pillow requests openai
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors peft scipy omegaconf
!pip install -q pyngrok

print("✓ Dependencies installed")

## 2. Clone Your Repository

In [ ]:
# Option A: Clone from GitHub
!git clone https://github.com/sapogeth/qylysh-higgsfiled.git
%cd qylysh-higgsfiled

# Option B: Upload files manually
# from google.colab import files
# uploaded = files.upload()  # Upload your project zip
# !unzip -q qylysh-higgsfiled.zip
# %cd qylysh-higgsfiled

print("✓ Repository loaded")

## 3. Configure for Colab (Update config for CUDA)

In [ ]:
# Create .env file with your OpenAI API key
import os
from getpass import getpass

# Optional: For GPT-based story generation
api_key = getpass("Enter your OpenAI API key (or press Enter to skip): ")
if api_key:
    with open('.env', 'w') as f:
        f.write(f'OPENAI_API_KEY={api_key}\n')
    print("✓ API key saved")
else:
    print("⚠️  No API key - will use fallback story generation")

# Update config.py for CUDA device
import torch
print(f"\n✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

## 4A. Mode 1: Run Full Flask Server with ngrok

In [ ]:
# Get ngrok auth token from: https://dashboard.ngrok.com/get-started/your-authtoken
from pyngrok import ngrok, conf
from getpass import getpass

ngrok_token = getpass("Enter your ngrok auth token: ")
conf.get_default().auth_token = ngrok_token

print("✓ ngrok configured")

In [ ]:
# Create a Colab-optimized app runner
with open('app_colab.py', 'w') as f:
    f.write('''
# Colab-optimized version of app.py
import sys
import os

# Override config for CUDA before imports
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Patch config.py for CUDA
import config
config.DEVICE = "cuda"
config.DTYPE = "float16"
config.LAZY_LOAD_MODEL = False  # Preload in Colab
config.ENABLE_TORCH_COMPILE = False  # Not needed in Colab

# Import the main app
from app import app, generator

# Force local mode in Colab
generator.use_local = True

if __name__ == '__main__':
    print("="*70)
    print("COLAB MODE: Using CUDA GPU for SDXL generation")
    print("="*70)
    app.run(host='0.0.0.0', port=5000, debug=False)
''')

print("✓ Colab runner created")

In [ ]:
# Start Flask server with ngrok tunnel
import threading
import time

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print("\n" + "="*70)
print("🌐 PUBLIC URL (share this):")
print(f"   {public_url}")
print("="*70)
print("\n⚠️  Keep this cell running to maintain the server")
print("   Open the URL above in your browser to use the app\n")

# Run Flask in background thread
def run_flask():
    os.system('python app_colab.py')

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Keep cell running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\nServer stopped")

## 4B. Mode 2: API-Only Mode (for Local Flask App)

Use this if you want to keep your Flask UI running locally but offload generation to Colab GPU.

In [ ]:
# Create a lightweight API server for remote generation
with open('colab_api.py', 'w') as f:
    f.write('''
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import torch

# Configure for CUDA
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import config
config.DEVICE = "cuda"
config.DTYPE = "float16"

from local_image_generator import LocalImageGenerator
from prompt_enhancer import PromptEnhancer

app = Flask(__name__)
CORS(app)  # Allow cross-origin requests from local app

# Initialize generator
print("Loading SDXL model on GPU...")
generator = LocalImageGenerator(lazy_load=False)
enhancer = PromptEnhancer()
print("✓ Model ready")

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'healthy',
        'device': config.DEVICE,
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'none'
    })

@app.route('/generate', methods=['POST'])
def generate():
    """Generate a single image from prompt"""
    data = request.get_json()
    prompt = data.get('prompt', '')
    negative_prompt = data.get('negative_prompt', config.NEGATIVE_PROMPT)
    
    # Optional IP-Adapter support
    ref_image_base64 = data.get('ref_image_base64')
    ip_adapter_scale = data.get('ip_adapter_scale', 0.6)
    
    ref_image = None
    if ref_image_base64:
        import base64
        from io import BytesIO
        from PIL import Image
        img_data = base64.b64decode(ref_image_base64)
        ref_image = Image.open(BytesIO(img_data)).convert('RGB')
    
    # Generate image
    image = generator.generate_single(
        prompt=prompt,
        negative_prompt=negative_prompt,
        ref_image=ref_image,
        ip_adapter_scale=ip_adapter_scale if ref_image else None
    )
    
    # Convert to base64
    from io import BytesIO
    import base64
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_base64 = base64.b64encode(buffered.getvalue()).decode()
    
    return jsonify({
        'success': True,
        'image_base64': img_base64
    })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)
''')

# Install CORS support
!pip install -q flask-cors

print("✓ API server created")

In [ ]:
# Start API server with ngrok
from pyngrok import ngrok
import threading
import time

# Start ngrok tunnel
public_url = ngrok.connect(5000)
api_url = f"{public_url}/generate"

print("\n" + "="*70)
print("🌐 COLAB API ENDPOINT:")
print(f"   {api_url}")
print("="*70)
print("\nAdd this to your local .env file:")
print(f"   COLAB_API_URL={public_url}")
print("\n⚠️  Keep this cell running to maintain the API")

# Run Flask in background
def run_api():
    os.system('python colab_api.py')

api_thread = threading.Thread(target=run_api, daemon=True)
api_thread.start()

# Keep running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\nAPI stopped")

## 5. Test the Setup

In [ ]:
# Quick test of local generation
import torch
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

from local_image_generator import LocalImageGenerator
gen = LocalImageGenerator(lazy_load=False)

test_prompt = "Aldar Kose walking across the golden steppe, 2D illustration"
print(f"\nGenerating test image...")

img = gen.generate_single(test_prompt)
img.save('test_output.png')

from IPython.display import Image, display
display(Image('test_output.png'))
print("\n✓ Generation successful!")